# Previsão de Preço com Rede Neural usando Indicadores Técnicos (RSI, SMA)

### 1. Coleta de Dados Históricos (5 anos)

In [1]:

import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Função para calcular RSI
def calculate_rsi(data, window=14):
    delta = data["Close"].diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Coleta de dados históricos
avax = yf.Ticker("AVAX-USD")
avax_data = avax.history(period="5y")

# Exibir os 5 primeiros registros
avax_data.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-07-13 00:00:00+00:00,4.613968,4.915115,4.481499,4.896294,28340,0.0,0.0
2020-07-14 00:00:00+00:00,4.909865,5.229564,4.762068,4.986754,3957,0.0,0.0
2020-09-22 00:00:00+00:00,4.986754,11.463443,4.125380,5.234632,288098840,0.0,0.0
2020-09-23 00:00:00+00:00,5.321654,5.329310,3.982604,4.118469,173091214,0.0,0.0
2020-09-24 00:00:00+00:00,3.817925,4.751878,3.539887,4.566561,96110964,0.0,0.0


### 2. Criação de Indicadores Técnicos

In [2]:

# Criação de Indicadores Técnicos
avax_data["RSI"] = calculate_rsi(avax_data)
avax_data["SMA_30"] = avax_data["Close"].rolling(window=30).mean()
avax_data["SMA_100"] = avax_data["Close"].rolling(window=100).mean()
avax_data["Price_Direction"] = np.where(avax_data["Close"].shift(-1) > avax_data["Close"], 1, 0)

# Exibir os 5 primeiros registros com indicadores
avax_data[["Close", "RSI", "SMA_30", "SMA_100", "Price_Direction"]].head()


,Close,RSI,SMA_30,SMA_100,Price_Direction
Date,,,,,
2020-07-13 00:00:00+00:00,4.896294,NaN,NaN,NaN,1
2020-07-14 00:00:00+00:00,4.986754,100.000000,NaN,NaN,1
2020-09-22 00:00:00+00:00,5.234632,100.000000,NaN,NaN,0
2020-09-23 00:00:00+00:00,4.118469,23.261447,NaN,NaN,1
2020-09-24 00:00:00+00:00,4.566561,41.334643,NaN,NaN,1


### 3. Preparação do Dataset

In [3]:

# Preparação do Dataset
avax_data.dropna(inplace=True)
features = avax_data[["RSI", "SMA_30", "SMA_100"]]
target = avax_data["Price_Direction"]

# Normalizar os dados
scaler = StandardScaler()
features = scaler.fit_transform(features)

features[:5]


array([[-0.31360323, -1.17631827, -1.20488229],
       [-0.6917322 , -1.1773514 , -1.2057129 ],
       [-0.80411503, -1.17836381, -1.20658258],
       [-0.11568954, -1.17895178, -1.20746686],
       [ 0.41288129, -1.17904003, -1.20765797]])

### 4. Divisão dos Dados

In [4]:

# Divisão dos Dados
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

X_train.shape, X_test.shape


((1088, 3), (273, 3))

### 5. Construção do Modelo de Rede Neural

In [6]:

# Construção do Modelo de Rede Neural
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

# Compilar o modelo
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Treinamento do modelo
model.fit(X_train, y_train, epochs=200, batch_size=10, verbose=1)


Epoch 1/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.5160 - loss: 0.6942 
Epoch 2/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step - accuracy: 0.5232 - loss: 0.6916
Epoch 3/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step - accuracy: 0.5213 - loss: 0.6899
Epoch 4/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 283us/step - accuracy: 0.4982 - loss: 0.6889
Epoch 5/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 271us/step - accuracy: 0.5236 - loss: 0.6874
Epoch 6/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 278us/step - accuracy: 0.5099 - loss: 0.6903
Epoch 7/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step - accuracy: 0.5079 - loss: 0.6906
Epoch 8/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 279us/step - accuracy: 0.5121 - loss: 0.6875
Epoch 9/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step - accuracy: 0.5592 - loss: 0.6859
Epoch 10/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step - accuracy: 0.5112 - loss: 0.6898
Epoch 11/200
109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step - accuracy: 0.5509 - loss: 0.6810
Epoch 12/200
109/1

### 6. Avaliação do Modelo

In [7]:

# Avaliação do Modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Acurácia do Modelo (Neural Network): {accuracy:.2f}")


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - accuracy: 0.4810 - loss: 0.7703
Acurácia do Modelo (Neural Network): 0.48


### 7. Previsão do Momento Atual

In [9]:

# Previsão do Momento Atual
current_data = avax_data[["RSI", "SMA_30", "SMA_100"]].iloc[-1].values.reshape(1, -1)
current_data_scaled = scaler.transform(current_data)
current_prediction = model.predict(current_data_scaled)
print("Momento Atual:", "Compra" if current_prediction > 0.5 else "Venda")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Momento Atual: Venda


/Users/luandemello/Workspace/2024/Atividade-Python/pondBitcoin/venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
